In [1]:
import sys
import os

import ase
from ase import Atoms
from ase.build import bulk, surface
from ase.io import read, write, Trajectory
from ase.optimize import LBFGS, BFGS, FIRE
from ase.constraints import FixAtoms, Hookean

import orb_models
from orb_models.forcefield import pretrained
from orb_models.forcefield.inference.calculator import ORBCalculator
from orb_models.forcefield.inference.d3_model import D3SumModel, AlchemiDFTD3

from atom_editor import AtomEditorUI

device = "cpu"  # or device="cuda"
orbff, atoms_adapter = pretrained.orb_v3_conservative_inf_omat(
  device=device,
  precision="float32-high",   # or "float32-highest" / "float64
)
orbff_d3 = D3SumModel(orbff, AlchemiDFTD3(functional="PBE", damping="BJ", compile=True))

calc = ORBCalculator(orbff_d3, atoms_adapter=atoms_adapter, device=device)
#calc = ORBCalculator(orbff, atoms_adapter=atoms_adapter, device=device)

Warp DeprecationWarning: The symbol `warp.vec` will soon be removed from the public API. Use `warp.types.vector` instead.


C:\Users\mmsoc\ASE_BASE\orb-models-main\orb-models-main\orb_models\common\training\util.py:20: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


In [3]:
import numpy as np
from dataclasses import dataclass
from typing import Optional
from ase import Atoms
from ase.geometry import get_distances


def random_rotation_matrix(rng: np.random.Generator) -> np.ndarray:
    """SO(3) 上で一様なランダム回転行列 (3x3) を返す。"""
    u1, u2, u3 = rng.random(3)
    q1 = np.sqrt(1 - u1) * np.sin(2 * np.pi * u2)
    q2 = np.sqrt(1 - u1) * np.cos(2 * np.pi * u2)
    q3 = np.sqrt(u1) * np.sin(2 * np.pi * u3)
    q4 = np.sqrt(u1) * np.cos(2 * np.pi * u3)  # (x,y,z,w) で w=q4

    x, y, z, w = q1, q2, q3, q4
    R = np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)],
    ])
    return R


@dataclass
class MoleculeSpec:
    molecule: Atoms
    n: int = 5
    min_height: float = 3.0
    max_height: float = 8.0
    min_dist: float = 2.0
    random_rotate: bool = True


def _prepare_molecule(molecule: Atoms) -> Atoms:
    """分子重心を原点にそろえたコピーを返す。"""
    mol0 = molecule.copy()
    mol0.positions -= mol0.get_center_of_mass()
    return mol0


def _try_place_one(
    atoms: Atoms,
    mol0: Atoms,
    spec: MoleculeSpec,
    top_z: float,
    cell,
    pbc,
    rng: np.random.Generator,
    max_trials: int,
) -> bool:
    """spec に従って mol0 を 1つ置けたら True。置けなければ False。"""
    for _trial in range(max_trials):
        # XY: セル内ランダム
        fx, fy = rng.random(2)
        shift_xy = fx * cell[0] + fy * cell[1]

        # 高さ: [min_height, max_height]
        h = rng.uniform(spec.min_height, spec.max_height)
        z = top_z + h

        mol = mol0.copy()

        # ランダム回転
        if spec.random_rotate:
            R = random_rotation_matrix(rng)
            mol.positions = mol.positions @ R.T

        # 平行移動
        mol.positions += shift_xy + np.array([0.0, 0.0, z])

        # 既存原子との衝突判定 (PBC+MIC)
        pos_exist = atoms.get_positions()
        pos_new = mol.get_positions()

        _, d = get_distances(pos_new, pos_exist, cell=cell, pbc=pbc)
        if d.size == 0:
            # atoms が空のとき等（通常 slab があるので不要だが安全のため）
            atoms += mol
            return True

        if d.min() >= spec.min_dist:
            atoms += mol
            return True

    return False


def add_two_random_molecules(
    slab: Atoms,
    spec_a: MoleculeSpec,
    spec_b: MoleculeSpec,
    max_trials: int = 1000,
    seed: Optional[int] = None,
    order: str = "AB",  # "AB" or "BA" で配置順を変えられる
) -> Atoms:
    """
    2種類の分子をランダム配置する。
    - spec_a, spec_b: 分子ごとの個数/高さ範囲/最小距離/回転の有無など
    - order: 先に置く分子の種類（混雑時は順序で成功率が変わる）
    """
    rng = np.random.default_rng(seed)

    atoms = slab.copy()
    cell = atoms.get_cell()
    pbc = atoms.get_pbc()
    top_z = slab.get_positions()[:, 2].max()

    mol0_a = _prepare_molecule(spec_a.molecule)
    mol0_b = _prepare_molecule(spec_b.molecule)

    specs = [(spec_a, mol0_a, "A"), (spec_b, mol0_b, "B")]
    if order.upper() == "BA":
        specs = specs[::-1]

    for spec, mol0, name in specs:
        for i in range(spec.n):
            ok = _try_place_one(
                atoms=atoms,
                mol0=mol0,
                spec=spec,
                top_z=top_z,
                cell=cell,
                pbc=pbc,
                rng=rng,
                max_trials=max_trials,
            )
            if not ok:
                print(
                    f"Warning: 分子{name} の {i+1} 個目を "
                    f"{max_trials} 回試行しても衝突なしで置けませんでした。"
                )

    return atoms

In [45]:
molecule = read("../molecule/.sdf")
molecule2 = read("../molecule/.sdf")
molecule3 = read("../molecule/.sdf")
molecule.calc = calc
molecule2.calc = calc
molecule3.calc = calc

opt = LBFGS(molecule)
opt.run(fmax=0.01)

opt = LBFGS(molecule2)
opt.run(fmax=0.01)

opt = LBFGS(molecule3)
opt.run(fmax=0.01)

       Step     Time          Energy          fmax
LBFGS:    0 21:51:46      -15.457695        1.330539
LBFGS:    1 21:51:46      -15.511313        0.543593
LBFGS:    2 21:51:46      -15.526421        0.327634
LBFGS:    3 21:51:46      -15.529469        0.129098
LBFGS:    4 21:51:46      -15.529690        0.023551
LBFGS:    5 21:51:46      -15.529700        0.002987
       Step     Time          Energy          fmax
LBFGS:    0 21:51:46      -18.154751        4.089063
LBFGS:    1 21:51:46      -18.307377        2.114542
LBFGS:    2 21:51:46      -18.364801        0.720683
LBFGS:    3 21:51:46      -18.376591        0.326149
LBFGS:    4 21:51:46      -18.378187        0.095600
LBFGS:    5 21:51:46      -18.378284        0.053728
LBFGS:    6 21:51:46      -18.378292        0.037284
LBFGS:    7 21:51:46      -18.378300        0.001600
       Step     Time          Energy          fmax
LBFGS:    0 21:51:46      -16.092094        2.244251
LBFGS:    1 21:51:46      -16.128799        3.123879

np.True_

In [2]:
slab = read("./st/.xyz")
ui = AtomEditorUI(slab)
ui.display()

C:\Users\mmsoc\anaconda3\envs\Orb-python-312\Lib\site-packages\nglview\__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
# molecule_A, molecule_B は ASE Atoms
specA = MoleculeSpec(molecule=molecule, n=1, min_height=3.0, max_height=3.0, min_dist=2.0, random_rotate=True)
specB = MoleculeSpec(molecule=molecule2, n=0,  min_height=3.0, max_height=3.0, min_dist=2.0, random_rotate=True)

atoms = add_two_random_molecules(
    slab=slab,
    spec_a=specA,
    spec_b=specB,
    max_trials=2000,
    seed=123,
    order="AB",
)

In [6]:
ui = AtomEditorUI(atoms)
ui.display()

In [53]:
num_iter = 5
molE = molecule.get_potential_energy()
slabE = slab.get_potential_energy() 

ad_st_list = []
addE_list=[]
for i in range(num_iter):
    # molecule_A, molecule_B は ASE Atoms
    specA = MoleculeSpec(molecule=molecule, n=1, min_height=3.0, max_height=3.0, min_dist=2.0, random_rotate=True)
    specB = MoleculeSpec(molecule=molecule2, n=0,  min_height=3.0, max_height=3.0, min_dist=2.0, random_rotate=True)
    
    atoms = add_two_random_molecules(
        slab=slab,
        spec_a=specA,
        spec_b=specB,
        max_trials=2000,
        order="AB",
    )
    
    atoms.calc = calc
    opt = LBFGS(atoms)
    opt.run(fmax=0.01)

    ad_st_list.append(atoms)
    atomsE = atoms.get_potential_energy()
    addE = atomsE - (molE+slabE)
    addE_list.append(addE)

       Step     Time          Energy          fmax
LBFGS:    0 17:23:01    -2793.645996        0.462046
LBFGS:    1 17:23:06    -2793.654785        0.385437
LBFGS:    2 17:23:11    -2793.661377        0.244977
LBFGS:    3 17:23:16    -2793.664307        0.170782
LBFGS:    4 17:23:22    -2793.668457        0.130631
LBFGS:    5 17:23:27    -2793.670410        0.117201
LBFGS:    6 17:23:32    -2793.672363        0.085941
LBFGS:    7 17:23:38    -2793.673584        0.096966
LBFGS:    8 17:23:45    -2793.674805        0.119401
LBFGS:    9 17:23:54    -2793.675781        0.110410
LBFGS:   10 17:24:02    -2793.676514        0.087076
LBFGS:   11 17:24:09    -2793.677734        0.083696
LBFGS:   12 17:24:16    -2793.679199        0.086246
LBFGS:   13 17:24:23    -2793.680176        0.079512
LBFGS:   14 17:24:30    -2793.681396        0.076720
LBFGS:   15 17:24:37    -2793.682373        0.074945
LBFGS:   16 17:24:44    -2793.683594        0.102371
LBFGS:   17 17:24:51    -2793.684814        0.10

In [67]:
for i,atoms in enumerate(ad_st_list):
    write(f"./output/add_st/.xyz", atoms)

In [63]:
ui = AtomEditorUI(ad_st_list[1])
ui.display()

In [55]:
addE_list

[np.float64(-0.6795282363891602),
 np.float64(-0.8662958145141602),
 np.float64(-0.7466669082641602),
 np.float64(-0.7810907363891602),
 np.float64(-0.7754755020141602)]

In [26]:
FS = ad_st_list[0].copy()

In [41]:
ui = AtomEditorUI(FS)
ui.display()

In [42]:
FS.calc = calc
opt = LBFGS(FS)
opt.run(fmax=0.01)

       Step     Time          Energy          fmax
LBFGS:    0 20:15:23    -2795.882080        8.789145
LBFGS:    1 20:15:28    -2797.066406        1.881731
LBFGS:    2 20:15:34    -2797.140137        2.539415
LBFGS:    3 20:15:39    -2797.205322        0.281642
LBFGS:    4 20:15:44    -2797.215332        0.247977
LBFGS:    5 20:15:50    -2797.234131        0.143150
LBFGS:    6 20:15:55    -2797.238037        0.131608
LBFGS:    7 20:16:00    -2797.245361        0.111928
LBFGS:    8 20:16:05    -2797.247803        0.127656
LBFGS:    9 20:16:11    -2797.252441        0.084564
LBFGS:   10 20:16:18    -2797.255371        0.076169
LBFGS:   11 20:16:25    -2797.257812        0.071830
LBFGS:   12 20:16:32    -2797.260498        0.097454
LBFGS:   13 20:16:39    -2797.263184        0.074995
LBFGS:   14 20:16:47    -2797.265381        0.056075
LBFGS:   15 20:16:54    -2797.267334        0.050673
LBFGS:   16 20:17:01    -2797.269043        0.085239
LBFGS:   17 20:17:08    -2797.271484        0.07

np.True_

In [43]:
ui = AtomEditorUI(FS)
ui.display()

In [44]:
FSE = FS.get_potential_energy()
ISE = ad_st_list[0].get_potential_energy()
print(FSE-ISE)

-3.320068359375
